In [ ]:
from astropy.constants import c
import astropy.coordinates as coord
import astropy.units as u
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('apw-notebook')
%matplotlib inline
from scipy.optimize import minimize

from comoving_rv.db import Session, Base, db_connect
from comoving_rv.db.model import (Run, Observation, TGASSource, SimbadInfo,
                                  SpectralLineInfo, SpectralLineMeasurement)

In [ ]:
db_path = '/Volumes/ProjectData/gaia-comoving-followup/db.sqlite'
engine = db_connect(db_path)
session = Session()

In [ ]:
Halpha, = session.query(SpectralLineInfo.wavelength).filter(SpectralLineInfo.name == 'Halpha').one()

For now, only get observations that are done that have a Simbad RV already in the database

In [ ]:
q = session.query(Observation).join(Run, SpectralLineMeasurement, SimbadInfo)
q = q.filter(Run.name == 'mdm-spring-2017')
q = q.filter(SpectralLineMeasurement.x0 != None)
q = q.filter((SimbadInfo.rv != None) & SimbadInfo.rv_qual.contains('A'))
q.distinct().count()

In [ ]:
observations = q.all()

In [ ]:
raw_offsets = np.zeros(len(observations)) * u.angstrom
all_sky_offsets = np.full((len(observations), 3), np.nan) * u.angstrom
true_rv = np.zeros(len(observations)) * u.km/u.s
night_id = np.zeros(len(observations), dtype=int)
obs_time = np.zeros(len(observations))
color_by = np.zeros(len(observations))

for i,obs in enumerate(observations):
    print(obs.object, obs.filename_1d)
    
    # color_by[i] = obs.airmass
    # color_by[i] = obs.exptime
    
    obs_time[i] = np.sum(np.array(list(map(float, obs.time_obs.split(':')))) / np.array([1., 60., 3600.]))
    color_by[i] = obs_time[i]
    night_id[i] = obs.night

    x0 = obs.measurements[0].x0 * u.angstrom
    offset = (x0 - Halpha)
    raw_offsets[i] = offset
    
    sky_offsets = []
    for j,meas in enumerate(obs.measurements[1:]):
        sky_offset = meas.x0*u.angstrom - meas.info.wavelength
        if meas.amp > 10 and meas.std_G < 2 and np.abs(sky_offset) < 4*u.angstrom:
            sky_offsets.append(sky_offset)
            all_sky_offsets[i,j] = sky_offset
            print(meas.info.name, meas.x0, meas.amp, sky_offset)
            
    sky_offsets = u.Quantity(sky_offsets)
    
    if len(sky_offsets) > 0:
        sky_offset = np.mean(sky_offsets)
    else:
        sky_offset = 0. * u.angstrom
        print("WARNING: not correcting with sky line")
    
    # sky_offset = -1.2*u.angstrom
    rv = (offset - sky_offset) / Halpha * c.to(u.km/u.s) + obs.v_bary
    true_rv[i] = obs.simbad_info.rv
    print(rv - obs.simbad_info.rv)
    print()

## Determine sky line weights

In [ ]:
def func(p, raw_offsets, all_sky_offsets, true_rv):
    weights = np.array(p)
    sky_offset = np.nanmean(weights * all_sky_offsets, axis=1)
    rv = (raw_offsets - sky_offset) / Halpha * c
    return np.nansum((rv - true_rv).to(u.km/u.s).value**2)

res = minimize(func, x0=np.ones(3), method='L-BFGS-B', 
               args=(raw_offsets, all_sky_offsets, true_rv))
res.x

In [ ]:
sky_offset = np.nanmean(res.x * all_sky_offsets, axis=1)
sky_offset[np.isnan(sky_offset)] = 0.*u.angstrom
corrected_rv = (raw_offsets - sky_offset) / Halpha * c.to(u.km/u.s)

### Try it per-night instead:

In [ ]:
corrected_rv = np.zeros(len(observations)) * u.km/u.s
night_weights = dict()
for n in np.unique(night_id):
    mask = night_id == n
    res = minimize(func, x0=np.ones(1), method='L-BFGS-B',
                   args=(raw_offsets[mask], all_sky_offsets[mask,:1], true_rv[mask]))
    weights = res.x
    
#     if n == 4:
#         # weights = np.array([1.2*0.94802129, 1.11843898, 0*0.49077822])
#         weights = np.array([1.2*0.94802129, 0*1.11843898, 0*0.49077822])
    print(weights)
    night_weights[n] = weights
    
    sky_offset = np.nanmean(weights * all_sky_offsets[mask,:1], axis=1)
    sky_offset[np.isnan(sky_offset)] = 0.*u.angstrom
    corrected_rv[mask] = (raw_offsets[mask] - sky_offset) / Halpha * c.to(u.km/u.s)

In [ ]:
fig,axes = plt.subplots(2, 5, figsize=(15,6.5), sharex='row', sharey='row')

_lim = (-400, 400)
_grid = np.linspace(_lim[0], _lim[1], 16)
_bins = np.linspace(-150, 150, 21)
for n in range(1,5+1):
    ax = axes[0, n-1]
    
    mask = night_id == n
    ax.scatter(corrected_rv[mask], true_rv[mask], c=color_by[mask], 
               marker='.', alpha=0.6, vmin=0, vmax=12)
    ax.plot(_grid, _grid, marker='', zorder=-10, color='#888888')
    
    # histogram
    ax = axes[1, n-1]
    drv = corrected_rv[mask] - true_rv[mask]
    ax.hist(drv, bins=_bins)
    
    print(n, 1.5 * np.median(np.abs(drv - np.median(drv))))
    
axes[0,0].set_xlim(_lim)
axes[0,0].set_ylim(_lim)
axes[1,0].set_xlim(_bins.min(), _bins.max())

fig.tight_layout()

In [ ]:
fig,axes = plt.subplots(3, 5, figsize=(15,8), sharex=True, sharey=True)

for n in range(1,5+1):
    ax = axes[:,n-1]
    mask = night_id == n
    
    for j in range(3):
        ax[j].scatter(obs_time[mask], all_sky_offsets[mask,j])
    
    ax[0].set_title('night {0}'.format(n))

axes[2,2].set_xlabel('time [hour]')
axes[0,0].set_ylabel('5577 offset')
axes[1,0].set_ylabel('6300 offset')
axes[2,0].set_ylabel('6364 offset')

fig.tight_layout()

Playing around: fit a GP

In [ ]:
from scipy.optimize import minimize
import celerite
from celerite import terms
from celerite.modeling import Model

In [ ]:
_night = 3
_sky_ix = 2
mask = (night_id==_night) & np.isfinite(all_sky_offsets[:, _sky_ix])

t = obs_time[mask]
y = all_sky_offsets[mask, _sky_ix].value
yerr = np.full_like(y, 0.3)

In [ ]:
class MeanModel(Model):
    parameter_names = ("const",)

    def get_value(self, t):
        return self.const

In [ ]:
mean_model = MeanModel(const=0.)

# Set up the GP model
kernel1 = terms.Matern32Term(log_sigma=np.log(np.median(np.abs(y-np.median(y)))), 
                             log_rho=np.log(0.05))
kernel2 = terms.Matern32Term(log_sigma=np.log(np.median(np.abs(y-np.median(y)))), 
                             log_rho=np.log(0.5))
kernel = kernel1 + kernel2

gp = celerite.GP(kernel, mean=mean_model, fit_mean=True)
gp.compute(t, yerr=yerr)
print("Initial log-likelihood: {0}".format(gp.log_likelihood(y)))

In [ ]:
gp.get_parameter_dict()

In [ ]:
# Define a cost function
def neg_log_like(params, y, gp):
    gp.set_parameter_vector(params)
    return -gp.log_likelihood(y)

# Fit for the maximum likelihood parameters
initial_params = gp.get_parameter_vector()
bounds = gp.get_parameter_bounds()
soln = minimize(neg_log_like, initial_params, 
                method="L-BFGS-B", bounds=bounds, args=(y, gp))
gp.set_parameter_vector(soln.x)
print("Final log-likelihood: {0}".format(-soln.fun))

# Make the maximum likelihood prediction
grid = np.linspace(t.min(), t.max(), 500)
mu, var = gp.predict(y, grid, return_var=True)
std = np.sqrt(var)

# Plot the data
color = "#ff7f0e"
plt.errorbar(t, y, yerr, marker='o', color="k", zorder=100, linestyle='none', alpha=0.5)
plt.plot(grid, mu, color=color)
plt.fill_between(grid, mu+std, mu-std, color=color, alpha=0.3, edgecolor="none")
plt.ylabel(r"$y$")
plt.xlabel(r"$t$")
plt.gca().yaxis.set_major_locator(plt.MaxNLocator(5))
plt.title("maximum likelihood prediction");

---

# Take a peek at velocity differences

In [ ]:
q = session.query(Observation.group_id).join(Run, SpectralLineMeasurement)
q = q.filter(Run.name == 'mdm-spring-2017')
print(q.distinct().count())
gids = np.array([gid for gid, in q.distinct().all()])

In [ ]:
rv_diffs = []
all_x0s = []
for gid in gids:
    meas_q = session.query(SpectralLineMeasurement).join(Observation, SpectralLineInfo)
    meas_q = meas_q.filter(SpectralLineInfo.name == 'Halpha')
    meas_q = meas_q.filter(Observation.group_id == gid)
    x0s = [meas.x0 for meas in meas_q.all()]
    
    if len(x0s) == 2:
        rv_diff = (x0s[1] - x0s[0])*u.angstrom / Halpha * c.to(u.km/u.s)
        rv_diffs.append(rv_diff)
        
        all_x0s = all_x0s + x0s
    
    else:
        rv_diffs.append(np.nan*u.km/u.s)
        
rv_diffs = u.Quantity(rv_diffs)
all_x0s = u.Quantity(all_x0s)

In [ ]:
# random shuffle
# TODO: redo this after correcting for barycenter, sky shifts, etc.
_derp = np.random.choice(len(all_x0s), size=len(all_x0s), replace=False)
random_rv_diff = (all_x0s - all_x0s[_derp])*u.angstrom / Halpha * c.to(u.km/u.s)

In [ ]:
_,bins,_ = plt.hist(rv_diffs[np.isfinite(rv_diffs)], bins=np.linspace(-150, 150, 31), 
                    normed=True, alpha=0.5)

plt.hist(np.random.normal(0, np.sqrt(25**2 + 25**2 + 5**2 + 5**2), size=64000), 
         bins=bins, normed=True, alpha=0.5);

_,bins,_ = plt.hist(random_rv_diff[np.isfinite(random_rv_diff)], bins=bins, normed=True, alpha=0.4)

Now look at differences in sky lines

In [ ]:
sky_diffs = []
for gid in gids:
    meas_q = session.query(SpectralLineMeasurement).join(Observation, SpectralLineInfo)
    meas_q = meas_q.filter(SpectralLineInfo.name == '[OI] 6300')
    meas_q = meas_q.filter(Observation.group_id == gid)
    x0s = [meas.x0 for meas in meas_q.all()]

    if len(x0s) == 2:
        sky_diff = (x0s[1] - x0s[0])*u.angstrom / (6300*u.angstrom) * c.to(u.km/u.s)
        sky_diffs.append(sky_diff)
    
    else:
        sky_diffs.append(np.nan*u.km/u.s)
    
sky_diffs = u.Quantity(sky_diffs)

In [ ]:
plt.hist(sky_diffs[np.isfinite(sky_diffs)], bins=np.linspace(-50, 50, 30));

TODO: ok, so apparently I need to correct for sky line shifts before doing RV difference??

In [ ]:
seps = []
for gid in gids[np.isfinite(rv_diffs) & (rv_diffs < 15*u.km/u.s)]:
    _q = session.query(TGASSource).join(Observation)
    _q = _q.filter(Observation.group_id == gid)
    tgas_data = _q.all()
    
    c1 = tgas_data[0].skycoord
    c2 = tgas_data[1].skycoord
    
    sep = c1.separation_3d(c2)
    seps.append(sep)

seps = u.Quantity(seps)

In [ ]:
plt.hist(seps[seps > 0*u.pc], bins='auto')